- Find the number of people thta re pure specialist versus the people that have different occupations
- Show if the association is random or not

In [2]:
import pandas as pd
data = pd.read_csv('../networks/data/global_before_1700.csv', index_col = [0])

print(len(set(data.wikidata_id)))


4266


In [3]:
len(set(data[data['meta_region']!='europe'].wikidata_id))

590

In [4]:
data.sort_values('birthyear', ascending=True).head(50)

,wikidata_id,individual_name,region_code,region_name,birthyear,meta_occupation,meta_region
712,Q1423663,Yajnavalkya,re_indian_world,Indian world,-800.0,astronomer | philosopher,non_europe
128,Q36303,Thales,re_greek_world,Greek World,-624.0,astronomer | philosopher | mathematician | phy...,non_europe
128,Q36303,Thales,re_greek_world,Greek World,-624.0,astronomer | philosopher | mathematician | phy...,antiquity
99,Q42458,Anaximander,re_greek_world,Greek World,-609.0,astronomer | geographer | philosopher | physic...,non_europe
99,Q42458,Anaximander,re_greek_world,Greek World,-609.0,astronomer | geographer | philosopher | physic...,antiquity
72,Q242270,Hecataeus of Miletus,re_greek_world,Greek World,-600.0,historian | geographer,antiquity
118,Q456560,Myia,re_greek_world,Greek World,-600.0,philosopher | mathematician,antiquity
60,Q298860,Hippasus,re_greek_world,Greek World,-600.0,musicologist | philosopher | mathematician,antiquity
79,Q235426,Theano,re_greek_world,Greek World,-600.0,astronomer | philosopher | mathematician,antiquity
60,Q298860,Hippasus,re_greek_world,Greek World,-600.0,musicologist | philosopher | mathematician,non_europe


In [5]:
data_europe = data[data['meta_region']=='europe']
print(len(data_europe))
len(set(data_europe.wikidata_id))/len(set(data.wikidata_id))

3676


0.8616971401781528

In [6]:
min(data_europe.birthyear)

501.0

In [7]:
len(set(data_europe[data_europe['birthyear']>=1500].wikidata_id))/len(data_europe)

0.8718715995647442

In [8]:
data_non_europe = data[data['meta_region']=='non_europe']
len(set(data_non_europe.wikidata_id))

590

In [9]:
duplicated_ids = data[data.duplicated(subset='wikidata_id', keep=False)]
duplicated_ids = duplicated_ids.sort_values('wikidata_id')
duplicated_ids.sample(10)

,wikidata_id,individual_name,region_code,region_name,birthyear,meta_occupation,meta_region
105,Q317942,Euhemerus,re_greek_world,Greek World,-400.0,historian | philosopher | anthropologist,non_europe
81,Q180671,Aratus,re_greek_world,Greek World,-314.0,astronomer,non_europe
27,Q206119,Marcus Terentius Varro,re_latin,Latin World,-115.0,musicologist | philosopher | linguist,antiquity
107,Q354377,Conon of Samos,re_greek_world,Greek World,-279.0,astronomer | mathematician,non_europe
35,Q928470,Simmias of Thebes,re_greek_world,Greek World,-500.0,philosopher | mathematician,antiquity
51,Q15091905,Deceneo,re_greek_world,Greek World,-100.0,astronomer | philosopher,antiquity
3,Q755297,Athenaeus Mechanicus,re_latin,Latin World,-199.0,philosopher | mathematician,antiquity
46,Q316334,Heraclides Ponticus,re_greek_world,Greek World,-384.0,astronomer | philosopher,non_europe
47,Q320132,Nearchus,re_greek_world,Greek World,-355.0,historian,antiquity
137,Q973972,Sporus of Nicaea,re_greek_world,Greek World,240.0,astronomer | linguist | mathematician,antiquity


In [10]:
df_specialist = data[['wikidata_id', 'meta_occupation']].drop_duplicates()

non_specialist = len(df_specialist[df_specialist['meta_occupation'].str.contains(' | ')])

non_specialist/len(df_specialist)

0.3764650726676043

In [11]:
df_specialist[df_specialist['meta_occupation'].str.contains(' | ')]

,wikidata_id,meta_occupation
1,Q723645,astronomer | mathematician
2,Q297515,historian | geographer
3,Q755297,philosopher | mathematician
4,Q706319,geographer | mathematician
5,Q82778,historian | philosopher | anatomist
...,...,...
962,Q31003843,theologian | logician
964,Q311495,philosopher | anatomist
967,Q10299689,historian | linguist | mathematician
969,Q167852,astronomer | philosopher | mathematician


In [12]:
df_specialist = df_specialist[~df_specialist['meta_occupation'].str.contains(' | ')]
len(set(df_specialist.wikidata_id))

2660

### Get the number of original occupations beofre cleaning


In [13]:
import sys
sys.path.append('../')
from functions.env import DATA_PATH,  DB_SCIENCE_PATH_NEW
import sqlite3

conn = sqlite3.connect(DB_SCIENCE_PATH_NEW)
data_occupation = pd.read_sql("SELECT * FROM individuals_occupation_information", conn)
print(len(set(data_occupation.individual_wikidata_id)))
len(set(data_occupation[data_occupation['birthyear']<=1700].individual_wikidata_id))

42668


7079

In [14]:



data_occupation = data_occupation.rename(
    columns={
        "occupations_name": "occupation",
        "individual_wikidata_id": "wikidata_id",
    }
)


data_occupation = data_occupation[['wikidata_id', 'occupation']].drop_duplicates()


data_occupation_filtered = data_occupation[data_occupation['wikidata_id'].isin(list(set(data.wikidata_id)))]
len(set(data_occupation_filtered.occupation))

273

In [17]:
data[data['region_name']=='Japan']

,wikidata_id,individual_name,region_code,region_name,birthyear,meta_occupation,meta_region
253,Q6538434,Sawaguchi Kazuyuki,re_japan,Japan,1670.0,mathematician,non_europe
256,Q786877,Shibukawa Shunkai,re_japan,Japan,1639.0,astronomer,non_europe
259,Q6538396,Andō Yūeki,re_japan,Japan,1624.0,astronomer,non_europe
265,Q11389078,Irumagawa Shigetsune,re_japan,Japan,1688.0,astronomer,non_europe
267,Q6538534,Muramatsu Shigekiyo,re_japan,Japan,1608.0,mathematician,non_europe
277,Q4067901,Konʼyō Aoki,re_japan,Japan,1698.0,botanist,non_europe
279,Q1150151,Abe no Seimei,re_japan,Japan,921.0,astronomer,non_europe
285,Q369158,Ogyū Sorai,re_japan,Japan,1666.0,philosopher | linguist | political scientist,non_europe
293,Q470531,Takakazu Seki,re_japan,Japan,1642.0,mathematician,non_europe
297,Q1136664,Gyōki,re_japan,Japan,668.0,geographer,non_europe


In [16]:
data.region_name.value_counts()

German world        890
Italy               678
France              608
British Islands     422
Low countries       345
Central Europe      251
Greek World         222
Spain               191
Arabic world        171
Nordic countries    167
Chinese world       121
Slav world           76
Persian world        73
Latin World          64
Indian world         33
Portugal             31
Japan                28
Balkans              17
South East Asia      10
Korea                 6
Ottoman Turkey        5
Name: region_name, dtype: int64